# Query 2

## Part 1

### Initialization

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Query 2") \
    .getOrCreate()

crime_data_path_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_data_path_2 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
129,application_1738075734771_0130,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Run this to load data from csv format

In [79]:
import time
start_time_csv = time.time()

crime_data_1 = spark.read.csv( \
    path=crime_data_path_1, \
    header=True, \
    inferSchema=True
).select("DATE OCC", "AREA NAME", "Status Desc")

crime_data_2 = spark.read.csv(
    path=crime_data_path_2,
    header=True,
    inferSchema=True
).select("DATE OCC", "AREA NAME", "Status Desc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Run this to load data from parquet format

In [73]:
import time
start_time_parquet = time.time()

# Define the path to the Parquet file on S3
s3_bucket_path = "s3://groups-bucket-dblab-905418150721/group34/Query_2/"

# Read the Parquet file into a Spark DataFrame
parquet_df = spark.read.parquet(s3_bucket_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Dataframe API

In [42]:
import time
start_time = time.time()

crimes = crime_data_1.union(crime_data_2)

# crimes.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 - The Window.partitionBy("Year").orderBy(col("Closed_Case_Rate").desc()) ensures that the ranking is applied within each year, ordering by Closed_Case_Rate in descending order.
 - dense_rank: Assigns ranks starting from 1 for each year, without skipping numbers in case of ties.
 - Column Addition: The .withColumn("Rank", dense_rank().over(window_spec)) adds the rank column.
 - Ordering: Finally, orderBy(col("Year"), col("Rank")) ensures the results are displayed sequentially by year and rank.

In [43]:
from pyspark.sql.functions import to_timestamp, col, year, count, when, sum, round
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank
import time

crimes = crimes \
    .withColumn("DATE OCC", to_timestamp(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a")) \
    .withColumn("Year", year(col("DATE OCC"))) \
    .drop("DATE OCC") \

crimes.show()

crimes_grouped = crimes \
    .groupBy("Year", "AREA NAME") \
    .agg(
        count("*").alias("Total_Crimes"), # Total crimes per year per police station
        sum(when(~((col("Status Desc") == "UNK") | (col("Status Desc") == "Invest Cont")), 1).otherwise(0))
        .alias("Closed_Crimes") # Count of closed crimes
    ) \
    .withColumn("Closed_Case_Rate", (col("Closed_Crimes") / col("Total_Crimes"))*100) \

window_spec = Window.partitionBy("Year").orderBy(col("Closed_Case_Rate").desc())

top_crimes = crimes_grouped \
    .drop("Total_Crimes", "Closed_Crimes") \
    .withColumn("Rank", dense_rank().over(window_spec)) \
    .filter(col("Rank") <= 3) \
    .orderBy(col("Year"), col("Rank"))

print("Final Result:")
top_crimes.show(100)

end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------+----+
|AREA NAME| Status Desc|Year|
+---------+------------+----+
|   Newton|Adult Arrest|2010|
|  Pacific| Invest Cont|2010|
|   Newton| Invest Cont|2010|
|Hollywood| Invest Cont|2010|
|  Central| Invest Cont|2010|
|  Central|Adult Arrest|2010|
|  Central| Invest Cont|2010|
|  Central|Adult Arrest|2010|
|  Central| Invest Cont|2010|
|  Central| Invest Cont|2010|
|  Central| Invest Cont|2010|
|  Central| Adult Other|2010|
|  Central|Adult Arrest|2010|
|  Central| Invest Cont|2010|
|  Central| Invest Cont|2010|
|  Central| Invest Cont|2010|
|  Central| Invest Cont|2010|
|  Central| Invest Cont|2010|
|  Central| Invest Cont|2010|
|  Central|Adult Arrest|2010|
+---------+------------+----+
only showing top 20 rows

Final Result:
+----+-----------+------------------+----+
|Year|  AREA NAME|  Closed_Case_Rate|Rank|
+----+-----------+------------------+----+
|2010|    Rampart| 32.84713448949121|   1|
|2010|    Olympic|31.515289821999087|   2|
|2010|     Harbor| 29.360

### SQL API

In [80]:
import time
start_time = time.time()

# Combine the datasets
crimes = crime_data_1.union(crime_data_2)

# Register the DataFrame as a temporary SQL table
crimes.createOrReplaceTempView("crimes")

# Display the table (optional)
# crimes.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
import time

# Step 1: Create formatted_crimes
spark.sql("""
    CREATE OR REPLACE TEMP VIEW formatted_crimes AS
    SELECT 
        TO_DATE(CAST(UNIX_TIMESTAMP(`DATE OCC`, 'MM/dd/yyyy hh:mm:ss a') AS TIMESTAMP)) AS `DATE OCC`,
        YEAR(TO_DATE(CAST(UNIX_TIMESTAMP(`DATE OCC`, 'MM/dd/yyyy hh:mm:ss a') AS TIMESTAMP))) AS Year,
        `AREA NAME`,
        `Status Desc`
    FROM crimes
""")

# formatted_crimes = spark.sql("SELECT * FROM formatted_crimes")
# formatted_crimes.show()

# Step 2: Create aggregated_crimes
spark.sql("""
    CREATE OR REPLACE TEMP VIEW aggregated_crimes AS
    SELECT 
        Year,
        `AREA NAME`,
        COUNT(*) AS Total_Crimes,
        SUM(CASE WHEN `Status Desc` NOT IN ('UNK', 'Invest Cont') THEN 1 ELSE 0 END) AS Closed_Crimes,
        Closed_Crimes * 100.0 / COUNT(*) AS Closed_Case_Rate
    FROM formatted_crimes
    GROUP BY Year, `AREA NAME`
""")

# aggregated_crimes = spark.sql("SELECT * FROM aggregated_crimes")
# aggregated_crimes.show()

# Step 3: Create ranked_crimes
spark.sql("""
    CREATE OR REPLACE TEMP VIEW ranked_crimes AS
    SELECT 
        Year,
        `AREA NAME`,
        Closed_Case_Rate,
        DENSE_RANK() OVER (PARTITION BY Year ORDER BY Closed_Case_Rate DESC) AS Rank
    FROM aggregated_crimes
""")

# ranked_crimes = spark.sql("SELECT * FROM ranked_crimes")
# ranked_crimes.show()

# Step 4: Execute final query
top_crimes = spark.sql("""
    SELECT 
        Year,
        `AREA NAME`,
        Closed_Case_Rate,
        Rank
    FROM ranked_crimes
    WHERE Rank <= 3
    ORDER BY Year, Rank
""")

# Display the result
print("Final result:")
top_crimes.show(100)

end_time = time.time()
print(f"Execution time for SQL API: {end_time - start_time:.2f} seconds")
print(f"Execution time for CSV import and SQL API: {end_time - start_time_csv:.2f} seconds")
# print(f"Execution time for Parquet import and SQL API: {end_time - start_time_parquet:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Final result:
+----+-----------+-----------------+----+
|Year|  AREA NAME| Closed_Case_Rate|Rank|
+----+-----------+-----------------+----+
|2010|    Rampart|32.84713448949121|   1|
|2010|    Olympic|31.51528982199909|   2|
|2010|     Harbor|29.36028339237341|   3|
|2011|    Olympic|35.04006009013520|   1|
|2011|    Rampart|32.49644718143060|   2|
|2011|     Harbor|28.51336246316431|   3|
|2012|    Olympic|34.29708533302119|   1|
|2012|    Rampart|32.46000463714352|   2|
|2012|     Harbor|29.50958584895668|   3|
|2013|    Olympic|33.58217940999398|   1|
|2013|    Rampart|32.10603829160530|   2|
|2013|     Harbor|29.72363895148855|   3|
|2014|   Van Nuys|32.02152352817050|   1|
|2014|West Valley|31.49754809505847|   2|
|2014|    Mission|31.22493985565357|   3|
|2015|   Van Nuys|32.26514067715784|   1|
|2015|    Mission|30.46376267367630|   2|
|2015|   Foothill|30.35300180365885|   3|
|2016|   Van Nuys|32.19451846212410|   1|
|2016|West Valley|31.40146437042384|   2|
|2016|   Foothill|29

## Part 2

In [37]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Combine Crime Datasets and Save to Parquet") \
    .getOrCreate()

# Define S3 bucket path for the final Parquet file
s3_bucket_path = "s3://groups-bucket-dblab-905418150721/group34/Query_2/"

# Define the paths for the two crime datasets
crime_data_path_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_data_path_2 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"

# Load both datasets (assuming CSV format; adjust for other formats if needed)
crime_df1 = spark.read.csv(crime_data_path_1, header=True, inferSchema=True)
crime_df2 = spark.read.csv(crime_data_path_2, header=True, inferSchema=True)

# Combine the datasets (ensure schemas are aligned before union)
combined_df = crime_df1.union(crime_df2)

# Save the combined dataset as a single Parquet file
combined_df.repartition(1).write \
    .mode("overwrite") \
    .parquet(s3_bucket_path)

print("Combined crime datasets successfully saved as a unique Parquet file to S3.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Combined crime datasets successfully saved as a unique Parquet file to S3.